In [1]:
import pandas as pd
import numpy as np
import sqlite3 

connection = sqlite3.connect('../data/vacancy.db')
cursor = connection.cursor()

def select(query):
    try:
        return pd.read_sql_query(query, connection)
    except:
        cursor.executescript(query)
        connection.commit()


In [10]:
query = r'''
    SELECT * 
    FROM vacancy
'''
select(query).head()

,id,name,premium,has_test,response_letter_required,area_id,salary_from,salary_to,salary_currency,salary_gross,type_id,metro_station_id,employer_id,published_at,schedule_id
0,10176972,Ведущий инженер-программист (Delphi),0,0,0,99,40000.0,NaN,RUR,NaN,open,None,136171.0,2021-01-13T08:21:14+0300,fullDay
1,19873107,IOS-разработчик,0,0,1,160,350000.0,1000000.0,KZT,0.0,open,None,40662.0,2020-12-28T06:52:32+0300,fullDay
2,20120535,Ведущий программист Scala / Java ( Senior Scal...,0,0,0,212,60000.0,90000.0,RUR,0.0,open,None,968814.0,2021-01-14T07:40:13+0300,fullDay
3,23473091,Java / Kotlin Developer,0,0,0,2,150000.0,NaN,RUR,0.0,open,15.215,1539356.0,2021-01-14T14:29:23+0300,flexible
4,23795691,Ведущий аналитик 1С,0,0,0,1,150000.0,NaN,RUR,0.0,open,9.142,115972.0,2021-01-11T18:45:51+0300,fullDay


In [3]:
query = r'''
    SELECT * 
    FROM vacancy
'''
select(query).info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5916 entries, 0 to 5915
Data columns (total 15 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   id                        5916 non-null   int64  
 1   name                      5916 non-null   object 
 2   premium                   5916 non-null   int64  
 3   has_test                  5916 non-null   int64  
 4   response_letter_required  5916 non-null   int64  
 5   area_id                   5916 non-null   int64  
 6   salary_from               5243 non-null   float64
 7   salary_to                 3445 non-null   float64
 8   salary_currency           5916 non-null   object 
 9   salary_gross              5910 non-null   float64
 10  type_id                   5916 non-null   object 
 11  metro_station_id          1323 non-null   object 
 12  employer_id               5905 non-null   float64
 13  published_at              5916 non-null   object 
 14  schedule

In [6]:
query = r'''
    SELECT *
    FROM schedule
'''
select(query)

,id,name
0,fullDay,Полный день
1,shift,Сменный график
2,flexible,Гибкий график
3,remote,Удаленная работа
4,flyInFlyOut,Вахтовый метод


In [9]:
query = r'''
    SELECT sc.name, COUNT(*) AS count_sc
    FROM vacancy AS vc
        JOIN schedule AS sc ON vc.schedule_id = sc.id
    GROUP BY vc.schedule_id
    ORDER BY count_sc DESC
'''
select(query)

,name,count_sc
0,Полный день,4187
1,Удаленная работа,1288
2,Гибкий график,379
3,Сменный график,57
4,Вахтовый метод,5


Напишите запрос, который посчитает количество вакансий по каждой валюте. Запрос должен вернуть два столбца — название валюты на русском и количество вакансий по ней. Отсортируйте по количеству вакансий по убыванию.

In [10]:
query = r'''
    SELECT * FROM currency
'''
select(query).head()

,code,abbr,name,rate
0,AZN,AZN,Манаты,0.023107
1,BYR,бел. руб.,Белорусские рубли,0.034966
2,EUR,EUR,Евро,0.011138
3,GEL,GEL,Грузинский лари,0.034400
4,KGS,KGS,Киргизский сом,1.131738


In [11]:
query = r'''
    SELECT 
        cr.name AS currency_name,
        COUNT(*) AS vacancy_count
    FROM vacancy vc
        JOIN currency cr ON vc.salary_currency = cr.code
    GROUP BY vc.salary_currency
    ORDER BY vacancy_count DESC
'''
select(query).head()

,currency_name,vacancy_count
0,Рубли,5228
1,Доллары,350
2,Тенге,168
3,Евро,56
4,Белорусские рубли,56


In [13]:
query = r'''
    SELECT * FROM employer
'''
select(query).head()

,id,name,trusted,open_vacancies
0,41,ЮНЭКТ,1.0,1.0
1,59,"РОЛЬФ, группа компаний",1.0,1.0
2,65,Московский аэропорт Домодедово,1.0,3.0
3,80,Альфа-Банк,1.0,3.0
4,124,Coleman Services,1.0,6.0


>Кто предлагает топ-5 самых высокооплачиваемых рублевых вакансий и о чем они?

In [5]:
query = r'''
    SELECT 
         em.name as employee_name,
         vc.name AS vacancy_name,
         salary_from AS salary
    FROM vacancy vc 
        JOIN employer em ON vc.employer_id = em.id
        JOIN currency cr ON vc.salary_currency = cr.code
    WHERE cr.name = 'Рубли'
    ORDER BY salary DESC
    LIMIT 5
'''
select(query)

,employee_name,vacancy_name,salary
0,АРЕС,Senior Frontend Engineer/ Архитектор трейдинго...,400000
1,АРЕС,Senior Backend Secure Engeneer ( информационна...,400000
2,LATOKEN,Chief Technology Officer (CTO),370000
3,Знаменка Капитал,"Team Lead (#C++, #Linux, #HPC)",350000
4,Набока Олеся Андреевна,Head of Analytics,320000


для каждой компании долю вакансий с гибким графиком от общего количества вакансий этой компании. Например:

    У компании А 30 открытых вакансий, из них 15 с гибким графиком. Доля = 15/30 = 0.5
    У компании Б 10 открытых вакансий, из них 9 с гибким графиком. Доля = 9/10 = 0.9

Назовем эту долю flexible_ratio.

Напишите запрос, который вернет компании с наибольшей flexible_ratio. Учитывайте только компании, у которых 10 и более открытых вакансий. Отсортируйте по убыванию flexible_ratio. Кто на третьем месте?

In [53]:
query = r'''
    WITH get_total AS (
    SELECT 
        em.name AS company_name,
        schedule_id,
        COUNT(*) OVER w AS total
    FROM vacancy vc
        JOIN employer em ON vc.employer_id = em.id
    WINDOW w AS (
        PARTITION BY vc.employer_id
    )), get_flexible_list AS (
    SELECT
        company_name,
        schedule_id,
        COUNT(*) * 1.0/total AS flexible,
        ROW_NUMBER() OVER w AS num
    FROM get_total
    WHERE schedule_id = 'flexible' AND total >= 10
    GROUP BY company_name, schedule_id
    WINDOW w AS (
        ORDER BY COUNT(*) * 1.0/total DESC
    )
    ORDER BY flexible DESC
    )
    SELECT
        company_name
    FROM get_flexible_list
    WHERE num = 3
'''
select(query)

,company_name
0,Атом Безопасность
